In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score
import pickle

In [2]:
# path = 'continuous_train.csv'
# data = pd.read_csv(path, sep = ',', na_filter = False)
data = pd.read_csv('databinarystudents.csv', sep = ',', encoding = 'UTF-8')
datos = data.set_index('id')
X_train = datos.drop(columns = ['yL']).copy()
y_train = data[['yL']].copy()

In [3]:
X_train.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32
id,,,,,,,,,,,,,,,,,,,,,
1,49.856,118.8,1,0,0,0,0,16.32096,0,18.762237,...,0.9709,0.0003,6,6,24.7138,0,0,7,0.056546,0.112860
2,71.836,34.8,0,0,0,0,1,15.00535,44,119.946790,...,0.4200,0.0000,8,4,38.0658,0,0,7,0.655260,0.886908
3,66.725,41.8,0,1,0,0,0,17.07023,27,10.879595,...,0.9650,0.0000,6,6,48.3807,0,1,4,0.067855,0.133944
4,41.696,132.0,0,0,1,0,0,16.31460,0,9.762927,...,0.0091,0.0001,6,4,59.8410,0,0,7,0.133230,0.260102
5,71.351,24.2,1,0,0,0,0,11.10790,31,151.581820,...,0.0051,0.0000,5,7,175.9250,0,0,3,0.318584,0.627729


In [32]:
def calculated_metrics(variablesSelected,X_trained_scaled,y_train,X_test_scaled,y_test):
    X_trained_scaled_aux = X_trained_scaled[variablesSelected].copy()
    X_test_scaled_aux = X_test_scaled[variablesSelected].copy()
    ## Regresion para cada cantidad de variables
    regr =  LogisticRegression(C=1, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    multi_class='warn', n_jobs=None,
                                    penalty='none', random_state=None, max_iter=500,
                                    solver='saga', tol=0.0001, verbose=0,
                                    warm_start=False)

    # Train the model using the training sets
    regr.fit(X_trained_scaled_aux,  y_train['yL'].values)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test_scaled_aux)

    y_pred = pd.DataFrame(y_pred, index=y_test.index, columns=["y_pred"])

#     total = y_pred.join(y_test)
    
    auc = roc_auc_score(y_test, y_pred, average="macro")
    #f1score = f1_score(y_test, y_pred, average="macro")
    accuracy = accuracy_score(y_test, y_pred)
#     total['y_menor_-1'] = (total['y']<-1).astype(int)
#     total['y_pred_menor_-1'] = (total['y_pred']<-1).astype(int)
#     total['good_class'] = (total['y_menor_-1']==total['y_pred_menor_-1']).astype(int)
#     mse = mean_squared_error(y_test, y_pred)
#     accuray =total['good_class'].sum()/len(total)
#     r2 = r2_score(y_test, y_pred)
    return auc,accuracy

In [33]:
modelToProof = [['x1','x2','x3','x4','x10','x11','x12','x14','x17','x18','x21','x22','x23','x24','x25','x27','x30','x32']]

In [34]:
modelToProof.append(['x1','x2','x3','x4','x10','x11','x12','x17','x21','x23','x25','x27'])

In [35]:
modelToProof.append(['x1','x2','x4','x10','x11','x12','x17','x21','x23','x25','x27'])

In [36]:
modelToProof.append(['x1','x2','x4','x10','x11','x12','x17','x23'])

In [37]:
modelToProof.append(['x1','x10','x12','x17','x23'])

In [39]:
resultados =  pd.DataFrame()
auc_total =  pd.DataFrame()
# mse_total =  pd.DataFrame()
accuracy_total =  pd.DataFrame()
for variablesTorun in modelToProof:
    print(variablesTorun)
    accuray = []
    auc = []
    if len(variablesTorun)>0:
        for j in range(1,1500):
            X_train_aux, X_test_aux, y_train_aux, y_test_aux = train_test_split(X_train, y_train, test_size=0.2)
            standarscaler = StandardScaler()
            X_trained_scaled_aux = standarscaler.fit_transform(X_train_aux)
            X_trained_scaled_aux = pd.DataFrame(X_trained_scaled_aux, index=X_train_aux.index, columns=X_train.columns.values)

            X_test_scaled_aux = standarscaler.transform(X_test_aux)

            X_test_scaled_aux = pd.DataFrame(X_test_scaled_aux, index=X_test_aux.index, columns=X_test_aux.columns.values)


            variablesSelected = list(variablesTorun)
            auc_aux,accuracy_aux= calculated_metrics(variablesSelected,X_trained_scaled_aux,y_train_aux,X_test_scaled_aux,y_test_aux)
            accuray.append(accuracy_aux)
            auc.append(auc_aux)

        auxRes= pd.DataFrame({'Modelo': [variablesTorun], 'Median: auc': [np.median(np.asarray(auc))],'Median: Accuray': [np.median(np.asarray(accuray))]})
        resultados = pd.concat([resultados,auxRes])
        name = ','.join(variablesTorun)
        auc_total[name] = auc
        accuracy_total[name] = accuray
#         break
#         resultados.to_csv('ResultadosCV_scores.csv')
    else:
        print(f"El conjunto de variables {variablesTorun} no tiene variables")
#     break
# resultado_scores_ranking_cv.reset_index(inplace = True,drop = True)

['x1', 'x2', 'x3', 'x4', 'x10', 'x11', 'x12', 'x14', 'x17', 'x18', 'x21', 'x22', 'x23', 'x24', 'x25', 'x27', 'x30', 'x32']
['x1', 'x2', 'x3', 'x4', 'x10', 'x11', 'x12', 'x17', 'x21', 'x23', 'x25', 'x27']


C:\ProgramData\Miniconda3\envs\celsia-neui\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


['x1', 'x2', 'x4', 'x10', 'x11', 'x12', 'x17', 'x21', 'x23', 'x25', 'x27']
['x1', 'x2', 'x4', 'x10', 'x11', 'x12', 'x17', 'x23']
['x1', 'x10', 'x12', 'x17', 'x23']


In [20]:
resultados.sort_values('Median: auc')

,Median: Accuray,Median: auc,Modelo
0,0.733333,0.650000,"[x1, x2, x4, x10, x11, x12, x17, x21, x23, x25..."
0,0.733333,0.666667,"[x1, x2, x4, x10, x11, x12, x17, x23]"
0,0.733333,0.674603,"[x1, x2, x3, x4, x10, x11, x12, x17, x21, x23,..."
0,0.733333,0.674641,"[x1, x10, x12, x17, x23]"
0,0.700000,0.675000,"[x1, x2, x3, x4, x10, x11, x12, x14, x17, x18,..."


In [21]:
resultado_total = auc_total.describe().T

In [22]:
resultado_total.sort_values(['std','50%'])

,count,mean,std,min,25%,50%,75%,max
"x1,x2,x4,x10,x11,x12,x17,x23",1499.0,0.666412,0.078043,0.395833,0.611111,0.666667,0.721591,0.940000
"x1,x10,x12,x17,x23",1499.0,0.673081,0.079423,0.365385,0.624434,0.674641,0.725000,0.980000
"x1,x2,x4,x10,x11,x12,x17,x21,x23,x25,x27",1499.0,0.648154,0.080219,0.375776,0.596591,0.650000,0.700925,0.954545
"x1,x2,x3,x4,x10,x11,x12,x17,x21,x23,x25,x27",1499.0,0.674898,0.082997,0.409091,0.624434,0.674603,0.728423,0.940000
"x1,x2,x3,x4,x10,x11,x12,x14,x17,x18,x21,x22,x23,x24,x25,x27,x30,x32",1499.0,0.671497,0.086106,0.346154,0.611111,0.675000,0.730159,0.944444


In [23]:
accuracy_total_org = accuracy_total.describe().T

In [24]:
accuracy_total_org.sort_values(['std','50%'], ascending= [True, False])

,count,mean,std,min,25%,50%,75%,max
"x1,x10,x12,x17,x23",1499.0,0.739449,0.069896,0.500000,0.700000,0.733333,0.800000,0.966667
"x1,x2,x4,x10,x11,x12,x17,x23",1499.0,0.731043,0.070600,0.533333,0.666667,0.733333,0.766667,0.966667
"x1,x2,x4,x10,x11,x12,x17,x21,x23,x25,x27",1499.0,0.713854,0.072896,0.433333,0.666667,0.733333,0.766667,0.933333
"x1,x2,x3,x4,x10,x11,x12,x17,x21,x23,x25,x27",1499.0,0.737781,0.075008,0.500000,0.700000,0.733333,0.800000,0.933333
"x1,x2,x3,x4,x10,x11,x12,x14,x17,x18,x21,x22,x23,x24,x25,x27,x30,x32",1499.0,0.712453,0.077151,0.366667,0.666667,0.700000,0.766667,0.966667


In [ ]:
%matplotlib notebook
accuracy_total.plot()

In [ ]:
%matplotlib notebook
accuracy_total.plot()

## Modelo Final

In [49]:
def training_model(variablesSelected,X_trained,y_train):
    X_trained_aux = X_trained[variablesSelected].copy()
    ## Regresion para cada cantidad de variables
    model_to_return = {}
    model_to_return['variableSelected'] = variablesSelected
    
    pipeline = Pipeline(steps = [('standarscaled',StandardScaler()),
                                 ('linearModel',LogisticRegression(C=1, class_weight=None, dual=False,
                                                                    fit_intercept=True, intercept_scaling=1,
                                                                    max_iter=500,
                                                                    multi_class='warn', n_jobs=None,
                                                                    penalty='none', random_state=None,
                                                                    solver='saga', tol=0.0001, verbose=0,
                                                                    warm_start=False))])


    # Train the model using the training sets
    pipeline.fit(X_trained_aux,  y_train['yL'].values)
    model_to_return['model'] = pipeline
    return model_to_return

In [50]:
variablesSelected = ['x1','x10','x12','x17','x23']

In [51]:
model_to_return = training_model(variablesSelected,X_train,y_train)

In [ ]:
model_to_return

In [ ]:
pickle.dump(model_to_return, open('modelBinary.sav', 'wb'))